### Importações

In [41]:
import pandas as pd
import pyodbc

### Criar conexão com o banco

In [42]:
driver = "ODBC Driver 18 for SQL Server"
server = "localhost"
database = "ETL_STAGE"
username = "sa"
password = "Admin@123"

connectionString = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=No;'
connection = pyodbc.connect(connectionString)

cursor = connection.cursor() 

### Inserir dados de distribuidora

In [43]:
try:
  df_distribuidora = pd.read_excel("DistribuidoraDeFilmes.xlsx")
  cursor.execute("DELETE FROM [DistribuidoraFilmes]")
  df_distribuidora.rename(columns={'Distribuidora de Filmes': 'DescricaoDistribuidora'}, inplace = True)
  for index, row in df_distribuidora.iterrows():
    cursor.execute("Insert into [DistribuidoraFilmes](IdDistribuidora,DescricaoDistribuidora)values(?,?)", int(row.name + 1), row.DescricaoDistribuidora)
  cursor.commit()
except Exception as e:
  cursor.rollback()
  print("Houve um erro ao inserir na tabela 'DistribuidoraFilmes'")
  raise e
finally:
  print("Dados inseridos na tabela 'DistribuidoraFilmes'")

Dados inseridos na tabela 'DistribuidoraFilmes'


### Inserir dados de Gênero

In [44]:
try:
  df_genero = pd.read_excel("Gêneros.xlsx")
  cursor.execute("DELETE FROM [GeneroFilmes]")
  df_genero.rename(columns={'Gênero': 'DescricaoGenero'}, inplace = True)
  for index, row in df_genero.iterrows():
    cursor.execute("Insert into [GeneroFilmes](IdGenero,DescricaoGenero)values(?,?)", int(index + 1), row.DescricaoGenero)
  cursor.commit()
except Exception as e:
  cursor.rollback()
  print("Houve um erro ao inserir na tabela 'GeneroFilmes'")
  raise e
finally:
  print("Dados inseridos na tabela 'GeneroFilmes'")

Dados inseridos na tabela 'GeneroFilmes'


### Inserir dados de Paises

In [45]:
try:
  df_pais = pd.read_excel("Paises.xlsx")
  df_pais.rename(columns={'País(es) produtor(es) da obra': 'DescricaoPaises'}, inplace = True)
  cursor.execute("DELETE FROM [Paises]")
  for index, row in df_pais.iterrows():
    cursor.execute("Insert into [Paises](IdPaises,DescricaoPaises)values(?,?)", int(index + 1), row.DescricaoPaises)
  cursor.commit()
except Exception as e:
  cursor.rollback()
  print("Houve um erro ao inserir na tabela 'Paises'")
  raise e
finally:
  print("Dados inseridos na tabela 'Paises'")

Dados inseridos na tabela 'Paises'


### Inserir dados da Tabela Principal

In [46]:
try:
  df_tabela_principal = pd.read_excel("TabelaPrincipal.xlsx")
  df_tabela_principal.drop(columns={"Ano de exibição", "CPB/ROE", "Origem da empresa distribuidora"}, inplace=True)
  df_tabela_principal.rename(columns={
    "Título da obra": "TituloFilme",
    "Gênero": "GeneroFilme",
    "País(es) produtor(es) da obra": "PaisFilme",
    "Nacionalidade da obra": "NacionalidadeFilme",
    "Data de lançamento": "DatadeLancamento",
    "Distribuidora": "DistribuidoraFilme",
    "Público no ano de exibição": "PublicoAnoExibicao",
    "Renda (R$) no ano de exibição": "RendaAnoExibicao"}, inplace=True)
  df_tabela_principal["DatadeLancamento"] = pd.to_datetime(df_tabela_principal["DatadeLancamento"], format=f'%d/%m/%Y', errors="coerce").dt.date
  cursor.execute("DELETE FROM [FilmesLancados]")
  for index, row in df_tabela_principal.iterrows():
    TituloFilme, GeneroFilme, PaisFilme, NacionalidadeFilme, DatadeLancamento, DistribuidoraFilme, PublicoAnoExibicao, RendaAnoExibicao = row
    cursor.execute(
       "Insert into [FilmesLancados](IdFilmesLancados,TituloFilme,GeneroFilme,PaisFilme,NacionalidadeFilme,DatadeLancamento,DistribuidoraFilme,PublicoAnoExibicao,RendaAnoExibicao)values(?,?,?,?,?,?,?,?,?)",
       int(index + 1), TituloFilme, GeneroFilme, PaisFilme, NacionalidadeFilme, DatadeLancamento, DistribuidoraFilme, PublicoAnoExibicao, RendaAnoExibicao)
  cursor.commit()
except Exception as e:
  cursor.rollback()  
  print("Houve um erro ao inserir na tabela 'FilmesLancados'")
  raise e
finally:
  print("Dados inseridos na tabela 'FilmesLancados'")

Dados inseridos na tabela 'FilmesLancados'


### Fechando conexões

In [47]:
cursor.close()
connection.close()
print("Conexão com o banco de dados encerrada")

Conexão com o banco de dados encerrada
